## 1. Import des librairies

In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import os
# print(os.listdir("../input"))
%matplotlib inline
sns.set({'figure.figsize':(16,8)})

## 2. Chargement et exploration du Dataset

In [ ]:
train_data=pd.read_csv("../input/train.csv")
test_data=pd.read_csv("../input/test.csv")
#train_data=pd.read_csv("/Users/mbp/Desktop/HETIC/KAGGLE/nyc-taxi-trip-duration/train.csv")
#test_data=pd.read_csv("/Users/mbp/Desktop/HETIC/KAGGLE/nyc-taxi-trip-duration/test.csv")

train_data.describe()

In [ ]:
train_data.head()

In [ ]:
print(test_data.shape)
print(train_data.shape)

## 3. Verification des doublons

In [ ]:
print(f"Data Frame a {train_data.shape[0]} lignes et {train_data.shape[1]} colonnes")
print(f"La colonne d'id a {train_data.id.nunique()} de la valeur identique")
train_data.duplicated().sum()

On n'a aucun doublon dans le Dataset

## 4. Verification des données manquantes

In [ ]:
train_data.isna().sum()

## 5. Analyse des outliers

In [ ]:
fig, ax = plt.subplots(7, sharex=True)
for i,c in enumerate(["vendor_id","passenger_count","pickup_longitude","pickup_latitude","dropoff_longitude","dropoff_latitude","trip_duration"]):
    sns.boxplot(train_data[c],ax=ax[i],width=1.5)
    ax[i].set_xscale("log")
    ax[i].set_xlabel("")
    ax[i].set_ylabel(c, fontsize=15,rotation=45)
fig.suptitle('Analyse des outliers', fontsize=20)


On a beaucoup de outliers dans des colonnes numériques, mais on ne pourrait pas les supprimer parce qu'ils influencent sur la prédiction du projet

- De plus 'Trip duration' en compte le plus et c'est la feature qu'on cherche à prédire



## 6. Conversion des données et attribution des variables

In [ ]:
train_data['pickup_datetime'] = pd.to_datetime(train_data['pickup_datetime'])
train_data['dropoff_datetime'] = pd.to_datetime(train_data['dropoff_datetime'])
train_data.info()

In [ ]:
train_data["store_and_fwd_flag"]=np.where(train_data["store_and_fwd_flag"] == "N",0,1)
train_data.head()

In [ ]:
X=train_data[['vendor_id',
       'passenger_count', 'pickup_longitude', 'pickup_latitude',
       'dropoff_longitude', 'dropoff_latitude']]
X.head()

In [ ]:
y=train_data['trip_duration']
y.head()

## 7. Train/Test Split

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_val_score;

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)
X_train.shape, y_train.shape, X_test.shape, y_test.shape


In [ ]:
rf = RandomForestRegressor(n_estimators=70)


## 8. Modeling

In [ ]:
rf.fit(X_train,y_train)

## 9. Prediction

In [ ]:
pred = rf.predict(X_test)
pred

In [ ]:
from sklearn.metrics import mean_squared_log_error 

In [ ]:
print(mean_squared_log_error(y_test,pred))

In [ ]:
submission =pd.read_csv("../input/sample_submission.csv")
#submission =pd.read_csv("/Users/mbp/Desktop/HETIC/KAGGLE/nyc-taxi-trip-duration/train.csv")
submission.head()

In [ ]:
test_pred = rf.predict(test_data[['vendor_id',
       'passenger_count', 'pickup_longitude', 'pickup_latitude',
       'dropoff_longitude', 'dropoff_latitude']])
print(test_pred)

In [ ]:
submission['trip_duration'] = test_pred

In [ ]:
submission.head()

In [ ]:
# my_submission.to_csv('/Users/mbp/Desktop/HETIC/KAGGLE/nyc-taxi-trip-duration/submission.csv', index=False)
submission.to_csv('submission_file.csv', index=False)